### Import thư viện

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Kiểm tra dữ liệu

In [2]:
data = pd.read_csv('../dataset/data.csv')
print(f"Kích thước dữ liệu: {data.shape}")
print(f"Số cột dữ liệu: {data.shape[1]}")
print(f"Số dòng dữ liệu: {data.shape[0]}")

print("\nMẫu dữ liệu:")
data.head()

Kích thước dữ liệu: (246823, 378)
Số cột dữ liệu: 378
Số dòng dữ liệu: 246823

Mẫu dữ liệu:


,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,panic disorder,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,panic disorder,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,panic disorder,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,panic disorder,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,panic disorder,1,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


### Phân tích dữ liệu

#### Kiểm tra giá trị null

In [3]:
null_counts = data.isnull().sum()
print(f"Tổng số giá trị null: {null_counts.sum()}")

Tổng số giá trị null: 0


#### Số lượng bệnh và triệu chứng có trong mẫu

In [19]:
disease_counts = data['diseases'].value_counts()

symptom_cols = data.columns[1:] 
symptom_counts = data[symptom_cols].sum().sort_values(ascending=False)

print("Số lượng bệnh: ", disease_counts.shape[0])
print("Số triệu chứng: ", symptom_counts.shape[0])

Số lượng bệnh:  721
Số triệu chứng:  377


#### Danh sách bệnh

In [ ]:
disease_list = disease_counts.index.tolist()
with open("disease_list.txt", "w", encoding="utf-8") as f:
    for disease in disease_list:
        f.write(f"{disease}\n")
disease_list

['cystitis',
 'vulvodynia',
 'nose disorder',
 'complex regional pain syndrome',
 'spondylosis',
 'peripheral nerve disorder',
 'esophagitis',
 'conjunctivitis due to allergy',
 'vaginal cyst',
 'hypoglycemia',
 'diverticulitis',
 'gastrointestinal hemorrhage',
 'acute bronchitis',
 'pneumonia',
 'sprain or strain',
 'infectious gastroenteritis',
 'fungal infection of the hair',
 'spontaneous abortion',
 'gout',
 'arthritis of the hip',
 'strep throat',
 'marijuana abuse',
 'injury to the arm',
 'liver disease',
 'bursitis',
 'acute pancreatitis',
 'spinal stenosis',
 'eczema',
 'noninfectious gastroenteritis',
 'benign prostatic hyperplasia (bph)',
 'injury to the leg',
 'acute bronchiolitis',
 'cholecystitis',
 'anxiety',
 'obstructive sleep apnea (osa)',
 'concussion',
 'dental caries',
 'sebaceous cyst',
 'problem during pregnancy',
 'chronic constipation',
 'sickle cell crisis',
 'psoriasis',
 'hyperemesis gravidarum',
 'depression',
 'pyogenic skin infection',
 'developmental dis

#### Danh sách triệu chứng

In [21]:
symptom_list = symptom_counts.index.tolist()
with open("symptom_list.txt", "w", encoding="utf-8") as f:
    for symptom in symptom_list:
        f.write(f"{symptom}\n")
symptom_list

['sharp abdominal pain',
 'vomiting',
 'headache',
 'cough',
 'sharp chest pain',
 'nausea',
 'back pain',
 'shortness of breath',
 'fever',
 'dizziness',
 'abnormal appearing skin',
 'nasal congestion',
 'leg pain',
 'skin swelling',
 'depressive or psychotic symptoms',
 'lower abdominal pain',
 'sore throat',
 'burning abdominal pain',
 'skin rash',
 'skin lesion',
 'arm pain',
 'weakness',
 'low back pain',
 'ear pain',
 'depression',
 'side pain',
 'diarrhea',
 'loss of sensation',
 'itching of skin',
 'skin growth',
 'neck pain',
 'abnormal involuntary movements',
 'insomnia',
 'anxiety and nervousness',
 'hip pain',
 'pelvic pain',
 'pain in eye',
 'chest tightness',
 'problems with movement',
 'diminished vision',
 'peripheral edema',
 'painful urination',
 'symptoms of eye',
 'retention of urine',
 'suprapubic pain',
 'difficulty breathing',
 'facial pain',
 'irregular appearing scalp',
 'acne or pimples',
 'knee pain',
 'hostile behavior',
 'coryza',
 'skin dryness, peeling, s

### Huấn luyện mô hình

#### Xử lý dữ liệu và chia tập train / test

In [12]:
X = data.iloc[:, 1:]  # Tất cả các cột trừ cột đầu tiên (diseases)
y = data.iloc[:, 0]   # Cột đầu tiên (diseases)

# Chuyển đổi nhãn bệnh thành số
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"Số lượng nhãn bệnh: {len(le.classes_)}")

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")


Số lượng nhãn bệnh: 721
Kích thước tập huấn luyện: (197458, 377)
Kích thước tập kiểm tra: (49365, 377)


#### Huấn luyện bằng nhiều mô hình

In [13]:
models = {
    'Random Forest': RandomForestClassifier(n_estimators=40, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

# Huấn luyện và đánh giá các mô hình
results = {}

for name, model in models.items():
    print(f"\nĐang huấn luyện mô hình {name}...")
    model.fit(X_train, y_train)
    
    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Độ chính xác của {name}: {accuracy:.4f}")
    
    # Lưu kết quả
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'y_pred': y_pred
    }


Đang huấn luyện mô hình Random Forest...
Độ chính xác của Random Forest: 0.8374

Đang huấn luyện mô hình Decision Tree...
Độ chính xác của Decision Tree: 0.8133


#### So sánh và chọn mô hình tốt nhất

In [17]:
# Tìm mô hình tốt nhất
best_model_name = max(results, key=lambda x: results[x]['accuracy'])
best_model = results[best_model_name]['model']
best_accuracy = results[best_model_name]['accuracy']
best_y_pred = results[best_model_name]['y_pred']

print(f"Mô hình tốt nhất: {best_model_name} với độ chính xác {best_accuracy:.4f}")

# Ma trận nhầm lẫn
print("\nMa trận nhầm lẫn:")
cm = confusion_matrix(y_test, best_y_pred)
print(cm)
    
# Báo cáo chi tiết với precision, recall, F1-score
print("\nBáo cáo chi tiết (Precision, Recall, F1-score):")
report = classification_report(y_test, best_y_pred, zero_division=1)
print(report)


Mô hình tốt nhất: Random Forest với độ chính xác 0.8374

Ma trận nhầm lẫn:
[[28  0  0 ...  0  0  0]
 [ 0 76  0 ...  0  0  0]
 [ 0  0 51 ...  0  0  0]
 ...
 [ 0  0  0 ...  3  0  0]
 [ 0  0  0 ...  0 87  0]
 [ 0  0  0 ...  0  0  1]]

Báo cáo chi tiết (Precision, Recall, F1-score):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        28
           1       0.94      0.94      0.94        81
           2       0.89      0.88      0.89        58
           3       1.00      1.00      1.00         4
           4       0.91      0.94      0.93        68
           5       0.56      0.83      0.67         6
           6       0.60      0.86      0.71         7
           7       0.67      0.71      0.69        17
           8       0.81      0.64      0.71        99
           9       0.70      0.74      0.72       182
          10       0.90      0.89      0.89       241
          11       0.69      0.68      0.69       243
          12     

#### Lưu mô hình

In [ ]:
# Tạo thư mục models nếu chưa tồn tại
os.makedirs('../models', exist_ok=True)

# Tối ưu lưu mô hình với nén
model_filename = f'../models/{best_model_name.replace(' ', '_').lower()}_model.joblib'
joblib.dump(best_model, model_filename, compress = 3) 

# Lưu label encoder cũng với nén
encoder_filename = '../models/label_encoder.joblib'
joblib.dump(le, encoder_filename, compress = 3)

print(f"Đã lưu mô hình tại: {model_filename}")
print(f"Đã lưu label encoder tại: {encoder_filename}")

Đã lưu mô hình tại: ../models/random_forest_model.joblib
Đã lưu label encoder tại: ../models/label_encoder.joblib
